<div style="text-align: center;">
      « ✦ —⋆——― ꒰ঌ·✦·໒꒱ ———⋆— ✦ »
</div>

 
<div style="text-align: center;">
<h1>Projeto final de Machine Learning</h1>
<h3> TRIO EPISTASIA - Bruna Guedes, Laura Medeiros e Mariana Melo</h2>
</div>
 

<div style="text-align: center;">
      « ✦ —⋆——― ꒰ঌ·✦·໒꒱ ———⋆— ✦ »
</div>

#### Sumário
 
1) Introdução
1) Pré processamento
2) Instanciação dos modelos e investigação dos conjuntos de hiperparametros
4) Comparação dos algoritmos com validação cruzada
5) Uso de ferramenta de explicação de modelos

# 1. Introdução

## 🧑‍💻 Vamos, então, para o Código Mágico?

O Código Mágico, capaz de resolver o desafio proposto, será construído a partir da linguagem Python, em conjunto com as suas bibliotecas `scikit-learn`, `xgboost`, `pandas` e `numpy`. O primeiro passo será, então, importar essas bibliotecas para o nosso ambiente de programação.

# 2. Revisão teórica

# 3. Metodologia

# 4. Desenvolvimento

## 4.1 Importação das bibliotecas

In [10]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import ExtraTreesClassifier

Vamos entender o que cada uma dessas bibliotecas faz?

* `pandas`: utilizada para a manipulação e organização dos dados em estruturas do tipo DataFrame, que permitem operações semelhantes às de planilhas, facilitando a seleção de atributos, filtragem de exemplos e preparação dos dados para os modelos.

* `NumPy`: responsável por operações numéricas eficientes, base para diversos cálculos matriciais e estatísticos. Sua utilização é essencial para lidar com arrays e fornecer suporte matemático às demais bibliotecas.

* `train_test_split` (scikit-learn): método empregado para dividir o conjunto de dados em subconjuntos de treino e teste, garantindo uma avaliação justa do desempenho dos algoritmos de aprendizado de máquina.

* `StandardScaler` é usado para padronizar features ao remover a média e dimensionando para a variância. Esse processo também é conhecido como normalização z-score [1]

A segunda etapa consiste em obter os dados a serem considerados para nosso estudo. Para nos ajudar nessa missão, o Mestre C4SS4R disponibilizou um grimório mágico (também conhecido como dataset `Glioma Grading Clinical and Mutation Features`). Vamos, então, carregar os dados desse dataset a partir do código abaixo.

## 4.2 Pré-processamento 

O pré-processamento é uma etapa crítica para a previsão de dados. Ele consiste em uma série de transformações aplicadas aos dados originais que os tornem mais limpos, adequados e otimizados, garantindo que as previsões feitas a partir deles tenham a maior acurácia possível. Esse tratamento pode consistir em remover dados de baixa qualidade/irrelevantes, converter dados para melhor entendimento do algoritmo, etc.

### 4.2.1 Obtenção dos dados

O conjunto de dados de `Glioma Grading Clinical and Mutation Features` contém 839 exemplos e 23 atributos reais, categoricos ou inteiros. Neste conjunto de dados, os 20 genes mutados com mais frequência e 3 características clínicas são considerados dos projetos de glioma cerebral TCGA-LGG e TCGA-GBM. A tarefa de previsão é determinar se um paciente é LGG ou GBM com determinadas características clínicas e moleculares/mutação. O objetivo principal é encontrar o subconjunto ideal de genes de mutação e características clínicas para o processo de classificação do glioma para melhorar o desempenho e reduzir custos [2]

In [2]:
df = pd.read_csv("TCGA_InfoWithGrade.csv")

#PRÉ-PROCESSAMENTO

# Selecionar linhas onde todos os valores são não-nulos
df_non_null_all = df[df.notnull().all(axis=1)]
print("\nDataFrame apenas com linhas onde todos os valores são non-null:")
print(df_non_null_all)

#


DataFrame apenas com linhas onde todos os valores são non-null:
     Grade  Gender  Age_at_diagnosis  Race  IDH1  TP53  ATRX  PTEN  EGFR  CIC  \
0        0       0             51.30     0     1     0     0     0     0    0   
1        0       0             38.72     0     1     0     0     0     0    1   
2        0       0             35.17     0     1     1     1     0     0    0   
3        0       1             32.78     0     1     1     1     0     0    0   
4        0       0             31.51     0     1     1     1     0     0    0   
..     ...     ...               ...   ...   ...   ...   ...   ...   ...  ...   
834      1       1             77.89     0     0     0     0     1     0    0   
835      1       0             85.18     0     0     1     0     1     0    0   
836      1       1             77.49     0     0     1     0     1     0    0   
837      1       0             63.33     0     0     1     0     0     0    0   
838      1       0             76.61     1  

### 4.2.2 Tipos dos dados


In [3]:
df_non_null_all.dtypes

Grade                 int64
Gender                int64
Age_at_diagnosis    float64
Race                  int64
IDH1                  int64
TP53                  int64
ATRX                  int64
PTEN                  int64
EGFR                  int64
CIC                   int64
MUC16                 int64
PIK3CA                int64
NF1                   int64
PIK3R1                int64
FUBP1                 int64
RB1                   int64
NOTCH1                int64
BCOR                  int64
CSMD3                 int64
SMARCA4               int64
GRIN2A                int64
IDH2                  int64
FAT4                  int64
PDGFRA                int64
dtype: object

Apenas uma das colunas do dataset é numérica: `Age_at_diagnosis`.

### 4.2.3 Escolha do target

In [4]:
FEATURES = [col for col in df_non_null_all.columns if col != 'Grade']
TARGET = ['Grade']

X = df_non_null_all[FEATURES]
y = df_non_null_all[TARGET]

X = X.values
y = y.values.ravel()  # o método `ravel` deixa os dados em 1 dimensão

### 4.2.4 Separação de dados de treino e teste

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Standard
scaler_standard = StandardScaler()

X_train_scaled_standard = scaler_standard.fit_transform(X_train)
X_test_scaled_standard = scaler_standard.transform(X_test)

print ("Número de dados para treino:", len(X_train_scaled_standard))
print ("Número de dados para teste:", len(X_test_scaled_standard))

Número de dados para treino: 587
Número de dados para teste: 252


## 4.3 Instanciação dos modelos

Tendo nossos dados preparados com target e features delimitados, podemos prosseguir para as etapas de instanciamento, previsão e validação dos modelos preditivos. Para esse projeto, escolhemos trabalhar com os cinco modelos abaixo: 

- RandomForestClassifier 
- GradientBoostClassifier (XGBoost)
- SupportVectorClassifier (SVC)
- Bernoulli NaiveBayes (NB)
- ExtraTreesClassifier 

### 4.3.1 RandomForestClassifier

O `RandomForestClassifier` realiza previsões por meio da criação de múltiplas **árvores de decisão**. Uma árvore de decisão segue uma estrutura semelhante a de um fluxograma, onde cada ramo (chamado de *nó de decisão*) simboliza um condicional. A criação dessas árvores envolve uma subamostra e um subconjunto de tributos aleatórios. Cada árvore é, portanto, exposta a um número diferente de recursos e a uma amostra diferente do conjunto de dados original [3], o que garante que uma única árvore não se atenha demais a um único subconjunto específico de dados, o que prejudicaria na acurácia das previsões.

Quando novos dados são apresentados ao modelo (nesse caso, um novo paciente com características singulares a respeito da idade e concentração de proteínas), ele é processado por todas as árvores de decisão individualmente. Cada árvore decide qual será o rótulo (LGG ou GBM), e ao final, o algoritmo considera a categoria com maior número de votos.  

Esse modelo toma como hiperparâmetros:

In [6]:
random_forest = RandomForestClassifier() 

random_forest.fit(X_train_scaled_standard, y_train)

previsao_RF = random_forest.predict(X_test_scaled_standard)

previsao_RF

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1])

### 4.3.2 XGBoostClassifier

O **Gradient Boosting** é uma forma de algoritmo que se beneficia do uso de uma equipe de outros modelos simples (geramente árvores de decisão) para prever dados. Em suas previsões, ele primeiro realiza um chute inicial simples (tirando a média ou moda, por exemplo), para então calcular os erros. Ele então treina modelos simples para tentar prever e compreender os erros que esse modelo inicial realizou, somando essa correção ao chute inicial e realizando uma nova previsão. Esse processo é repetido várias vezes, até que os erros sejam minimizados ao máximo. Isso lhe da um controle excelente sobre problemas de *overfitting*.

O `XGBoost` (Extreme Gradient Boosting) é uma versão otimizada desse algoritmo, implementando melhorias que o tornam mais preciso, como a regularização L1 e L2, que adiciona uma penalidade para modelos que se demonstram muito apegados a um conjunto específico dos dados e poderia gerar previsões "viciadas". Usamos o `XGBoostClassifier` para problemas de classificação, como é o nosso caso.

In [7]:
xgboost = XGBClassifier()

xgboost.fit(X_train_scaled_standard, y_train)

previsao_XGB = xgboost.predict(X_test_scaled_standard)

previsao_XGB

array([0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1])

### 4.3.3 SupportVectorClassification (SVC)



O algoritmo `SupportVectorClassification` é uma modalidade do `SupportVectorMachine` (SVM). Ele busca manter rótulos diferentes o mais distantes possíveis - ou seja, ele tenta separar e maximizar a margem entre os dados da melhor maneira possível, tornando-os distínguiveis o suficiente para uma previsão com boa acurácia. Os pontos mais próximos entre a margem que divide os dados são chamados de **vetores de suporte**, o que justifica o nome do algoritmo. O SVC obtêm um espaço onde a divisão é pode ser realizada por meio de um plano, utilizando a ténica **Kernel**: ele transforma os dados para um espaço de maior dimensão (3D ou mais), assim conseguindo separar os dados linearmente. O SVC então calcula o **produto escalar** para definir o Hiperplano de Separação (fronteira de decisão). 

Sendo um classificador discriminativo, ele é ideal para problemas de classificação binária, e portanto podemos aplicá-lo para o nosso caso:

In [8]:
SVC = SVC()

SVC.fit(X_train_scaled_standard, y_train)

previsao_SVC = SVC.predict(X_test_scaled_standard)

previsao_SVC

array([0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1])

### 4.3.4 Bernoulli NaiveBayes (NB)

O classificador Naive Bayes (NB) é um algoritmo de aprendizado de máquina que se baseia no **Teorema de Bayes** para realizar classificações. Ele calcula a **probabilidade a posteriori** de cada classe, dadas as características de um novo dado. O algoritmo então atribui o dado à classe com a maior probabilidade. O Teorema de Bayes é dado pela equação:

$$
P(A | B) = \frac{P(A) P(B | A).}{P(B)}.
$$
*(Onde $P(A)$ é a probabilidade inicial; $P(B|A)$ é a verossimilhança (probabilidade de $A$ dado que $B$ ocorreu); $P(B)$ é a probabilidade individual (também chamada de evidência); e $P(A|B)$ é a probabilidade a posteriori)*

O termo "Naive" (Ingênuo) vem da principal simplificação que o algoritmo faz: ele assume que todas as características (features) são condicionalmente independentes umas das outras. Essa suposição, entre tanto, o torna um algoritmo muito funcional para previsões.

O `Bernoulli NaiveBayes` é uma variante desse classificador que analisa a presença ou ausência de uma *feature* - sendo melhor aplicado quando os dados estão em sua forma binária (0 ou 1). Em nosso dataset, ele calcula a probabilidade de cada classe dado a presença ou ausência das mutações. 

In [9]:
BNB = BernoulliNB()

BNB.fit(X_train_scaled_standard, y_train)

previsao_BNB = BNB.predict(X_test_scaled_standard)

previsao_BNB

array([0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0])

### 4.3.5 ExtraTreesClassifier 

O `ExtraTreesClassifier` (Extremely Randomized Trees) segue a mesma premissa do `RandomForestClassifier`, como discutido anteriormente, porém, possui caracteristicas que ajudam a minimizar ainda mais o overfitting e ajudar na generalização. A principal delas é que ele introduz mais aleatoriedade, tornando as árvores mais diversas ao também randomizar o split de dados (diferente do RFC, que escolhe o melhor ponto de corte usando métricas como Entropia). Devido a esse fato, ele também pode executar de forma bem mais rápida comparado ao RFC tradicional. 

In [11]:
ETC = ExtraTreesClassifier()

ETC.fit(X_train_scaled_standard, y_train)

previsao_ETC = ETC.predict(X_test_scaled_standard)

previsao_ETC

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1])

# Resultados e Discussões

# Conclusões

# Referências Bibliográficas

[1] SCIKIT-LEARN. StandardScaler. Disponível em: <https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html>.

[2] UCI Machine Learning Repository. Disponível em: <https://archive.ics.uci.edu/dataset/759/glioma+grading+clinical+and+mutation+features+dataset>.

[3] https://www.datacamp.com/pt/tutorial/random-forests-classifier-python

https://www.datacamp.com/pt/tutorial/guide-to-the-gradient-boosting-algorithm
